In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import tensorflow as tf
import lime
import time
import shutil
import random
import csv
import os,sys
from skimage.io import imreadcd 
from skimage.segmentation import mark_boundaries
%load_ext autoreload
%autoreload 2

#main_dir = os.path.normpath(os.getcwd() + os.sep + os.pardir + os.sep + os.pardir)
main_dir = os.getcwd()